# 01. 농작물 재배 추천 지역 시각화

전국 2,072개 행정동의 토양 데이터를 기반으로 농작물별 최적 재배 지역을 Folium 지도로 시각화합니다.

## 주요 기능

| 기능 | 설명 |
|------|------|
| **농작물 → 지역** | 특정 농작물이 잘 자라는 전국 최적 지역 Top N 검색 |
| **지역 → 농작물** | 특정 지역에서 잘 자라는 추천 농작물 Top N 검색 |

## 데이터 출처

- **토양 적합도**: 농촌진흥청 흙토람 (Selenium 크롤링)
- **행정동 경계**: 통계청 GeoJSON

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from src import (
    CropRegionVisualizer,
    create_crop_map,
    SOIL_COLUMNS,
)

## 1. 데이터 로드

In [ ]:
# 시각화 객체 생성 및 데이터 로드
viz = CropRegionVisualizer()
viz.load_data()

print(f'총 행정동 수: {len(viz.gdf):,}개')
print(f'토양 데이터 수: {len(viz.soil_data):,}개')
print(f'병합 데이터 수: {len(viz.merged_data):,}개')

In [ ]:
# 사용 가능한 농작물 목록
crops = viz.get_available_crops()
print(f'총 {len(crops)}개 농작물')
print(crops[:20], '...')

In [ ]:
# 사용 가능한 지역 목록 (샘플)
regions = viz.get_available_regions()
print(f'총 {len(regions):,}개 지역')
regions.head(10)

## 2. 농작물 → 최적 재배 지역 검색

특정 농작물을 입력하면 전국에서 해당 농작물이 잘 자라는 지역 Top N을 반환합니다.

In [ ]:
# 사과 최적 재배 지역 Top 30
apple_regions = viz.search_by_crop("사과", top_n=30)

print(f'사과 최적 재배 지역 Top 30')
print('=' * 50)
apple_regions[['adm_nm', 'sidonm', '면적당 최적지', '면적당 적지', '최적지_점수']].head(10)

In [ ]:
# 사과 최적 재배 지역 지도
apple_map = viz.create_map(
    apple_regions,
    title="사과 최적 재배 지역 Top 30",
    show_markers=True,
    show_boundaries=True
)
apple_map

In [ ]:
# 감귤 최적 재배 지역 (제주 집중)
gyul_regions = viz.search_by_crop("감귤", top_n=30)

gyul_map = viz.create_map(
    gyul_regions,
    title="감귤 최적 재배 지역 Top 30"
)
gyul_map

In [ ]:
# 배추 최적 재배 지역
baechu_regions = viz.search_by_crop("배추", top_n=30)

baechu_map = viz.create_map(
    baechu_regions,
    title="배추 최적 재배 지역 Top 30"
)
baechu_map

## 3. 지역 → 추천 농작물 검색

특정 지역(시도 + 동)을 입력하면 해당 지역에서 잘 자라는 농작물 Top N을 반환합니다.

In [ ]:
# 경상북도 안동 지역 추천 농작물
andong_crops = viz.search_by_region("경상북도", "안동", top_n=15)

print('경상북도 안동 지역 추천 농작물 Top 15')
print('=' * 50)
andong_crops[['작물이름', '면적당 최적지', '면적당 적지', '최적지_점수']].head(15)

In [ ]:
# 전라남도 목포 지역 추천 농작물
mokpo_crops = viz.search_by_region("전라남도", "목포", top_n=15)

print('전라남도 목포 지역 추천 농작물 Top 15')
print('=' * 50)
mokpo_crops[['작물이름', '면적당 최적지', '면적당 적지', '최적지_점수']].head(15)

In [ ]:
# 강원도 춘천 지역 추천 농작물
chuncheon_crops = viz.search_by_region("강원특별자치도", "춘천", top_n=15)

print('강원도 춘천 지역 추천 농작물 Top 15')
print('=' * 50)
chuncheon_crops[['작물이름', '면적당 최적지', '면적당 적지', '최적지_점수']].head(15)

## 4. 토양 성분 분석

각 지역의 토양 성분(pH, 유기물, 유효인산 등)을 확인합니다.

In [ ]:
# 사과 최적 재배 지역의 토양 성분 평균
# 존재하는 컬럼만 선택
available_soil_cols = [col for col in SOIL_COLUMNS if col in apple_regions.columns]

if available_soil_cols:
    print('사과 최적 재배 지역 토양 성분 평균')
    print('=' * 50)
    apple_regions[available_soil_cols].mean().round(2)

## 5. 지도 저장

In [ ]:
# 사과 지도 저장
filepath = viz.save_map(apple_map, "map_apple.html")
print(f'지도 저장 완료: {filepath}')

In [ ]:
# 간편 함수로 지도 생성 및 저장
# onion_map = create_crop_map("양파", top_n=30, save=True)

## 6. 결론

### 주요 발견

| 농작물 | 주요 최적 지역 |
|--------|---------------|
| 사과 | 경상북도 (안동, 영주, 청송) |
| 감귤 | 제주특별자치도 |
| 배추 | 강원도 (평창, 정선), 전라남도 |
| 양파 | 전라남도 (무안, 함평) |

### 토양 성분과 작물 적합성

- **산도(pH)**: 대부분의 작물은 pH 5.5~7.0에서 최적
- **유기물**: 높을수록 토양 비옥도 향상
- **유효인산**: 뿌리 발달과 개화에 중요

### 데이터 수집 과정의 어려움

1. **대규모 크롤링**: 64개 농작물 × 전국 읍면동 = 32만+ 파라미터 조합
2. **시간 소요**: 1만개 파라미터 수집에 1.5시간, 데이터 변환에 5.5시간
3. **해결 전략**: 33개 CSV 파일로 분할 저장 후 병합